In [1]:
from skillcorner.client import SkillcornerClient

import pandas as pd

import numpy as np

import os

secret_password = os.getenv("mdp_skillcorner")
client = SkillcornerClient(username = "Nathan.talbot@etu.uca.fr", password = secret_password)

In [14]:
data_import1 = client.get_physical(params = {"data_version" : 3, 'competition_edition': 549, "average_per" : "p30otip",
                                            "possession" : "otip", "physical_check_passed" : "true"})

In [12]:
data_import2 = client.get_physical(params = {"data_version" : 3, 'competition_edition': 549, "average_per" : "p30tip", "possession" : "tip",
                                             "physical_check_passed" : "true"})

In [16]:
data_import3 = client.get_physical(params = {"data_version" : 3, 'competition_edition': 549, "physical_check_passed" : "true"})

In [17]:
data1 = pd.DataFrame(data_import1)
data2 = pd.DataFrame(data_import2)
data3 = pd.DataFrame(data_import3)

In [34]:
data4 = data3.merge(data2, on = ['player_name', 'player_short_name', 'player_id', 'player_birthdate', 'team_name', 'team_id', 'match_name', 'match_id', 'match_date', 'competition_name', 'competition_id', 'season_name', 'season_id', 'competition_edition_id', 'position', 'position_group', 'physical_check_passed'])

In [35]:
data4.to_excel("test.xlsx")